### 설명
- 하나손해보험 웹페이지 크롤링
- 보험 최근 약관만 다운로드
- 코드 정리

## 1. 모듈 임포트

In [ ]:
from selenium import webdriver  # 브라우저를 제어하는 라이브러리
from selenium.webdriver.common.by import By  # HTML 요소를 찾을 때 사용
from selenium.webdriver.common.keys import Keys  # 키보드 입력을 자동화할 때 사용
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests

## 2. 크롬 브라우저로 사이트 열기

In [ ]:
# Chrome 브라우저 실행
driver = webdriver.Chrome()
driver.get("https://www.hanainsure.co.kr/w/disclosure/product/saleProduct")
time.sleep(1)

## 3. 태그를 찾고 클릭 & 약관 pdf 다운로드

In [ ]:
# 보험 상품 선택 -> 보험 상품 목록 -> 판매 기간 -> 약관 보기 -> pdf 다운로드
# (1) 자동차보험 -> (2) 5가지(개인용/업무용/등) -> (3) n개(보험상품목록) -> (4) 현재 -> (5) 약관보기 -> (6) pdf창 -> (7) 다운로드 -> (8) 원래 창으로 이동


# (1) 자동차보험 선택
button = driver.find_element(By.XPATH, f"//a[@data-ins_type='0' and text()='자동차보험']")
driver.execute_script("arguments[0].click();", button)
time.sleep(0.5)

# (2) 5가지 상품 목차 순회
for j in range(1,6):
    button = driver.find_element(By.XPATH, f"//a[@data-ins_type='0' and @data-ins_dtl_type='0{j}']")
    driver.execute_script("arguments[0].click();", button)
    time.sleep(0.5)

    # (3) links로 받아서 순회
    links = driver.find_elements(By.XPATH, "//div[@id='divStep02']//a")
    for i, link in enumerate(links):
        name = link.text        # 보험상품 이름 저장
        driver.execute_script("arguments[0].click();", link)
        time.sleep(0.1)

        # (4) 현재
        button = driver.find_element(By.XPATH, "//div[@id='divStep03']//a[1]")
        driver.execute_script("arguments[0].click();", button)
        time.sleep(0.1)

        # (5) 약관보기
        button = driver.find_element(By.XPATH, "//div[@class='btn_group']/a[1]")
        driver.execute_script("arguments[0].click();", button)

        # (6) pdf창 페이지로 이동
        all_tabs = driver.window_handles    # 모든 탭을 리스트형태로
        new_tab = all_tabs[1]               # 새 탭(리스트의 두번째)로 선택
        driver.switch_to.window(new_tab)    # 새 탭으로 이동
        time.sleep(1)

        # (7) 다운로드
        pdf_url = driver.current_url
        response = requests.get(pdf_url)
        file_name = f"./하나손해보험 약관파일/1_자동차보험/{j}_{i+1}_{name}_pdf.pdf"
        with open(file_name, "wb") as f:
            f.write(response.content)
            time.sleep(0.1)
        
        # (8) 두번째 창을 닫고 원래 창으로 이동
        driver.close()  # 현재 탭 닫기
        driver.switch_to.window(all_tabs[0])  # 다시 원래 첫 번째 탭으로 돌아가기

# 종료
driver.quit()